<a href="https://colab.research.google.com/github/JoaoVitorSantiagoNogueira/case-brick-seguros/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extração
Aqui nós extraimos os dados da internet dos carros

In [242]:
import requests
import re
import pandas as pd

from threading import Thread

## Extrair todos os cógidos FIPE

Feito manualmente, todos os códigos FIPE estão presentes na pagina:
https://www.tabelafipebrasil.com/fipe/carros
bastou fazer um regex no formato [\d]+-[\d]


arquivo gerado ficou diisponível em "https://github.com/JoaoVitorSantiagoNogueira/case-brick-seguros/blob/main/FIPE-CODES.txt"

In [ ]:
def get_fipe_codes():
  codes_string = requests.get("https://raw.githubusercontent.com/JoaoVitorSantiagoNogueira/case-brick-seguros/main/FIPE-CODES.txt").text
  codes_list = re.split("\r\n",codes_string)
  return codes_list

In [ ]:
len(get_fipe_codes())

6893

In [ ]:
get_fipe_codes()[922:933]

['004166-1',
 '004167-0',
 '004168-8',
 '004169-6',
 '004171-8',
 '004172-6',
 '004173-4',
 '004174-2',
 '004175-0',
 '004176-9',
 '004177-7']

## Pegar Informações de Veiculo

In [ ]:
# https://brasilapi.com.br/api/fipe/preco/v1/038003-2  Últimos números são o código fipe

def get_json_from_fipe(Codigo_FIPE, query = ""):

  if query != "":
    query = "?tabela_referencia="+query

  response = requests.get("https://brasilapi.com.br/api/fipe/preco/v1/" + Codigo_FIPE + query)
  if response.status_code == 200:
    return response.json()


In [ ]:
#sample
type(get_json_from_fipe("038003-2", "9"))

list

#Cria a tabela inteira de um veiculo
Navegando preenchendo mês-a-mês, no formato mm-aa

## Cria o histórico de preços

In [ ]:
df = pd.json_normalize(get_json_from_fipe("038003-2", "10"))
df

,valor,marca,modelo,anoModelo,combustivel,codigoFipe,mesReferencia,tipoVeiculo,siglaCombustivel,dataConsulta
0,"R$ 18.742,00",Acura,Integra GS 1.8,1992,Gasolina,038003-2,março de 2006,1,G,"segunda-feira, 13 de maio de 2024 18:54"
1,"R$ 17.197,00",Acura,Integra GS 1.8,1991,Gasolina,038003-2,março de 2006,1,G,"segunda-feira, 13 de maio de 2024 18:54"


### Metodo 1, threading

In [ ]:
def generate_full_fipe_entry(codigo_fipe, resultados, numero_tread):
    new_line = get_json_from_fipe(codigo_fipe)
    #completar com outros anos de dados da tabela fipe
    print("tread " + str(codigo_fipe) + " has started")
    i=0
    entrada_atual = new_line[0]["mesReferencia"]
    ultima_entrada = None
    while (ultima_entrada != entrada_atual):
      i += 1
      serie_t.status_code == 200:
      if type(serie_temporal) == list :
        ultima_entrada = serie_temporal[0]["mesReferencia"]
        for modelo_atual in new_line:
          for modelo_antigo in serie_temporal:
            if (modelo_atual["anoModelo"] == modelo_antigo["anoModelo"] and
               modelo_atual["siglaCombustivel"] == modelo_antigo["siglaCombustivel"]):
               modelo_atual[modelo_antigo["mesReferencia"]] = modelo_antigo["valor"]
               break

    resultados [numero_tread] = new_line
    print("tread " + str(codigo_fipe) + " is done")
    return

In [ ]:
#processo demorado, em torno de 1 minuto por veiculo
def create_whole_dataset(codigos, num_treads = 10):
  #criar um dataframe e atualizar eles é custoso
  #https://stackoverflow.com/questions/13784192/creating-an-empty-pandas-dataframe-and-then-filling-it
  #fazemos primeiro em dicts e depois criamos o data frame
  data_set = []

  threads = [None] * num_treads
  results = [None] * num_treads

  offset= 0 #usado para poder loopar o numero de treads

  created_thread_count = 0
  for i in range(len(threads)): ## cada linha gera sua pesquisa própria
    if i+offset >= len (codigos):
      break
    created_thread_count += 1
    threads[i] = Thread(target = generate_full_fipe_entry, args = (codigos[i+offset], results, i))
    threads[i].start()


  for i in range(created_thread_count):
    threads[i].join()

  return results

In [ ]:
result = create_whole_dataset(['004166-1',
 '004167-0',
 '004168-8',
 '004169-6',
 '004171-8',
 '004172-6',
 '004173-4',
 '004174-2',
 '004175-0',
 '004176-9',
 '004177-7'
])
df = pd.json_normalize(result)
df

KeyboardInterrupt: 

In [ ]:
#criar tabelas com força bruta

cods = get_fipe_codes()

i=0
while i < len(cods):
  if len(cods) < i+10:
    offset = len(cods) - i
  else:
    offset=50
  dt = create_whole_dataset(cods[i:i+offset], num_treads = offset)
  dt = ds_normalizer(dt)
  save_csv_to_drive(dt, str(i)+"to"+str(i+offset))
  i+=offset




tread 038002-4 has started
tread 038001-6 has started
tread 060002-4 has startedtread 060001-6 has started

tread 038003-2 has started
tread 060004-0 has startedtread 060005-9 has started
tread 060003-2 has startedtread 060007-5 has started
tread 060006-7 has started


tread 060008-3 has started
tread 060010-5 has started
tread 060009-1 has startedtread 060015-6 has started

tread 060016-4 has startedtread 060013-0 has started
tread 006001-1 has started
tread 060012-1 has started

tread 060014-8 has started
tread 060011-3 has startedtread 006005-4 has started

tread 006008-9 has started
tread 006002-0 has startedtread 006004-6 has started
tread 006003-8 has started

tread 006006-2 has started
tread 006007-0 has started
tread 006010-0 has started
tread 006011-9 has started
tread 006013-5 has started
tread 006014-3 has startedtread 006009-7 has started

tread 006016-0 has startedtread 006015-1 has started

tread 037001-0 has started
tread 037002-9 has started
tread 006017-8 has started
t

Exception in thread Thread-265 (generate_full_fipe_entry):
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 537, in _make_request
    response = conn.getresponse()
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 461, in getresponse
    httplib_response = super().getresponse()
  File "/usr/lib/python3.10/http/client.py", line 1375, in getresponse
    response.begin()
  File "/usr/lib/python3.10/http/client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "/usr/lib/python3.10/http/client.py", line 287, in _read_status
    raise RemoteDisconnected("Remote end closed connection without"
http.client.RemoteDisconnected: Remote end closed connection without response

During handling of the above exception, another exception occurred:

Tr

KeyboardInterrupt: 

### Método 2 ThreadPoolExecutor

In [15]:
import requests
from concurrent.futures import ThreadPoolExecutor

def get_url(url):
    return requests.get(url)

def get_url_batch(list_of_urls):
  with ThreadPoolExecutor(max_workers=50) as pool:
      return list(pool.map(get_url,list_of_urls))

In [20]:
def FIPE_url_list_generator(codes, query):

  base_url = "https://brasilapi.com.br/api/fipe/preco/v1/"
  Q = []*4

  all_urls = []
  for c, q in zip(codes, query):
    if q == "":
      q=0
    for i in range(0,4):
      all_urls.append(base_url+c+"?tabela_referencia="+str(q+i))

  return all_urls

In [22]:
test_url_list = FIPE_url_list_generator(['004166-1',
 '004167-0',
 '004168-8',
 '004169-6',
 '004171-8',
 '004172-6',
 '004173-4',
 '004174-2',
 '004175-0',
 '004176-9',
 '004177-7'
], [10, 30, 20, 90, 100, 200, 13, 18, 22, 10, 30])

## Salvar DF em CSV no drive

In [27]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [28]:
def ds_normalizer():
  r = []
  for item in result:
    r += item
  df = pd.json_normalize(r)
  return df

In [29]:
def save_csv_to_drive(csv, filename):
  csv.to_csv('/content/drive/My Drive/' + filename + ".csv", index= False)

## Transformar JSONs em uma base de dados e Préprocessar

In [ ]:
def flatten_extend(matrix):
    flat_list = []
    for row in matrix:
      flat_list.extend(row)
    return flat_list

In [111]:
def group_jsons_by_field(jsons, fields):
  database_lines = []
  while jsons:
    ref = jsons[0]

    #primeira passada, encontra todos os elementos que batem
    match_list = []
    for i in range(0, len(jsons)):
      values_match =  True
      for field in fields:
        values_match = values_match and field in jsons[i] and jsons[i][field] == ref[field]

      if values_match:
        match_list.append(i)


    #remove as coincidências e junta tudo em uma tabela só
    matches = []
    for i in reversed(match_list):
      matches.append(jsons.pop(i))
    #junta elas numa entrada só
    database_lines.append(matches)
  return database_lines

In [249]:
def merge_json_split_months_costs(jsons):
  new_keys = []
  new_values = []
  for json in jsons:
      new_keys.append(json.pop("mesReferencia"))
      new_values.append(json.pop("valor"))

  if len(new_keys)<4: ##ignora entradas sem a devida quantidade de valores. Poderiamos preencher, mas por simplicidade e para não enviezar os dados façamos isso.
    return

  #trocar para valor numérico no formato aaaa/mm
  months = ["janeiro de ","fevereiro de ","março de ","abril de ", "maio de ", "junho de ", "julho de ", "agosto de ", "setembro de ", "outubro de ", "novembro de ", "dezembro de "]
  month_numbers = ["01", "02", "03", "04", "05", "06", "07", "08", "09", "10", "11", "12"]

  for key in range(0, len(new_keys)):
    for month_text, month_number in zip (months, month_numbers):
      new_keys[key] = new_keys[key].replace(month_text, month_number)

  new_keys = [i[2:-1]+i[:2] for i in new_keys]

  #ordena por data
  results = sorted(zip(new_keys, new_values))
  new_keys   = [i[0]for i in results]
  new_values = [i[1]for i in results]

  #atribui as datas como, mesDesejado , o valor target do Mes desejado, e o valor dos 3 mêses anteriores, se vazio, se faltar um dos 3 completa pela média dos demais
  jsons[0]["mesDesejado"] = new_keys  [3]
  jsons[0]["target"]      = int (re.sub("[^\d]", "", new_values[3]))//100
  jsons[0]["mes-1"]       = int (re.sub("[^\d]", "", new_values[2]))//100
  jsons[0]["mes-2"]       = int (re.sub("[^\d]", "", new_values[1]))//100
  jsons[0]["mes-3"]       = int (re.sub("[^\d]", "", new_values[0]))//100

  return jsons[0]

In [250]:
sample_json_list = [j.json() for j in get_url_batch(test_url_list)]

In [251]:
#processo de limpeza, planificar todos os itens, agrupar código fipe iguais, préprocessar os dados
sample_json_list = flatten_extend(sample_json_list )
sample_json_list = [item for item in sample_json_list if type(item) == dict]  #remover entradas que voltaram com erro
sample_json_list = group_jsons_by_field(sample_json_list,["codigoFipe", "modelo", "anoModelo" , "siglaCombustivel"])
sample_json_list = [merge_json_split_months_costs(item) for item in sample_json_list]
sample_json_list = [item for item in sample_json_list if type(item) == dict] #remover entradas que voltaram com erro

[{'marca': 'GM - Chevrolet',
  'modelo': 'Sonoma CE 4.3 V6',
  'anoModelo': 1995,
  'combustivel': 'Gasolina',
  'codigoFipe': '004166-1',
  'tipoVeiculo': 1,
  'siglaCombustivel': 'G',
  'dataConsulta': 'terça-feira, 14 de maio de 2024 20:08',
  'mesDesejado': '200606',
  'target': 18232,
  'mes-1': 18269,
  'mes-2': 18165,
  'mes-3': 18063},
 {'marca': 'GM - Chevrolet',
  'modelo': 'Sonoma CE 4.3 V6',
  'anoModelo': 1994,
  'combustivel': 'Gasolina',
  'codigoFipe': '004166-1',
  'tipoVeiculo': 1,
  'siglaCombustivel': 'G',
  'dataConsulta': 'terça-feira, 14 de maio de 2024 20:08',
  'mesDesejado': '200606',
  'target': 15513,
  'mes-1': 15710,
  'mes-2': 15558,
  'mes-3': 15567},
 {'marca': 'GM - Chevrolet',
  'modelo': 'Syclone 5.7 V8',
  'anoModelo': 1992,
  'combustivel': 'Gasolina',
  'codigoFipe': '004167-0',
  'tipoVeiculo': 1,
  'siglaCombustivel': 'G',
  'dataConsulta': 'terça-feira, 14 de maio de 2024 20:08',
  'mesDesejado': '200712',
  'target': 15656,
  'mes-1': 15733,
 

## Transfere para o pandas

## Cria a rede neural

In [8]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn.functional as F

In [4]:
class basic_car_price_predictor(nn.Module):
  def __init__(self):
      super(basic_car_price_predictor, self).__init__()

    #numero de canais: 3 preços dos meses anteriores, 1 ano de fabricação, 1 ranking da marca, 1 tipo de combustivel
      self.layer1 = nn.Sequential(
          nn.Conv2d(in_channels= 6, out_channels=32, kernel_size=3, padding=1),
          nn.BatchNorm2d(32),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2)
      )

      self.layer2 = nn.Sequential(
          nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          nn.MaxPool2d(2)
      )

      self.fc1 = nn.Linear(in_features=64*6*6, out_features=600)
      self.drop = nn.Dropout2d(0.25)
      self.fc2 = nn.Linear(in_features=600, out_features=120)
      self.fc3 = nn.Linear(in_features=120, out_features=10)

  def forward(self, x):
      out = self.layer1(x)
      out = self.layer2(out)
      out = out.view(out.size(0), -1)
      out = self.fc1(out)
      out = self.drop(out)
      out = self.fc2(out)
      out = self.fc3(out)

      return out


### Configurar GPU

In [9]:
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

False


### Criar Modelo Base

In [12]:
error = nn.CrossEntropyLoss()

learning_rate = 0.001

In [14]:
BasicPredictor = basic_car_price_predictor()

print(BasicPredictor)
total_params = sum(p.numel() for p in BasicPredictor.parameters())
print(f"Number of parameters: {total_params}")


BasicPredictor.to(device)

BasicOptimizerCNN = torch.optim.Adam(BasicPredictor.parameters(), lr=learning_rate)


basic_car_price_predictor(
  (layer1): Sequential(
    (0): Conv2d(6, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=2304, out_features=600, bias=True)
  (drop): Dropout2d(p=0.25, inplace=False)
  (fc2): Linear(in_features=600, out_features=120, bias=True)
  (fc3): Linear(in_features=120, out_features=10, bias=True)
)
Number of parameters: 1476778


prepara os datasets para serem lidos

In [ ]:
train_set =
test_set =

batchSize = 100

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batchSize)
test_loader  = torch.utils.data.DataLoader(test_set , batch_size=batchSize)

In [ ]:
iteration_list, loss_list, accuracy_list, predictions_list, labels_list = trainModel(BasicPredictor, optimizerCNN, 5)

In [11]:
def trainModel(model, optimizer, num_epochs):
  count = 0
  # Lists for visualization of loss and accuracy
  loss_list = []
  iteration_list = []
  accuracy_list = []

  # Lists for knowing classwise accuracy
  predictions_list = []
  labels_list = []
  for epoch in range(num_epochs):
      for images, labels in train_loader:
          # Transfering images and labels to GPU if available
          images, labels = images.to(device), labels.to(device)

          train = Variable(images.view(100, 1, 28, 28))
          labels = Variable(labels)

          # Forward pass
          outputs = model(train)
          loss = error(outputs, labels)

          # Initializing a gradient as 0 so there is no mixing of gradient among the batches
          optimizer.zero_grad()

          #Propagating the error backward
          loss.backward()

          # Optimizing the parameters
          optimizer.step()

          count += 1

      # Testing the model

          if not (count % 50):    # It's same as "if count % 50 == 0"
              total = 0
              correct = 0

              for images, labels in test_loader:
                  images, labels = images.to(device), labels.to(device)
                  labels_list.append(labels)

                  test = Variable(images.view(100, 1, 28, 28))

                  outputs = model(test)

                  predictions = torch.max(outputs, 1)[1].to(device)
                  predictions_list.append(predictions)
                  correct += (predictions == labels).sum()

                  total += len(labels)

              accuracy = correct * 100 / total
              loss_list.append(loss.data)
              iteration_list.append(count)
              accuracy_list.append(accuracy)

          if not (count % 500):
              print("Iteration: {}, Loss: {}, Accuracy: {}%".format(count, loss.data, accuracy))
  return iteration_list, loss_list, accuracy_list, predictions_list, labels_list